# Compliance Overview
This notebook showcases LUSID's powerful and flexible compliance engine, enabling firms to define, execute, and monitor investment rules tailored to their specific mandates.

### Key Capabilities
- **Custom Rule Templates** - Modular templates allow firms to define reusable compliance logic — such as filtering assets, grouping by attributes, or applying custom thresholds.
- **Flexible Parameterization** - Rules can accept dynamic inputs (e.g. rating thresholds, sector filters), allowing the same logic to be applied across different portfolios or strategies.
- **Derived Metrics Support** - Supports complex calculations like weighted average credit ratings using derived properties (e.g. PV-weighted scores).
- **Post-Trade & Pre-Trade Compliance** - Compliance checks can be run against live holdings or pending orders, enabling proactive risk controls before execution.

### Workflow Summary
We begin by creating all the necessary mock financial data — including instruments, properties, portfolios, and a valuation recipe — all within a dedicated custom scope to avoid conflicts with existing data in other environments.

Once the setup is complete, we move into the compliance workflow, which includes:
1. Define Templates & Rules: Create custom rule logic using composable steps and dynamic parameters.
2. Run Compliance Checks: Evaluate portfolios against rules and capture detailed breakdowns.
3. Analyze Results: Identify warnings or breaches with full lineage and contributing factors.
4. Pre-Trade Checks: Simulate portfolio impact of new orders and ensure compliance before trading.

LUSID's compliance framework gives asset managers the tools to build transparent, auditable, and adaptable rules — aligned with their investment philosophy, client mandates, and regulatory obligations.

## Imports

This section loads all the necessary Python modules and LUSID SDK components, enabling 
authentication/configuration setup so that all necessary APIs are initialized and ready to use.

In [ ]:
import lusid
import os
from lusid.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)

import lusid.api as la
import pandas as pd
import lusid.models as lm
import datetime
from datetime import datetime, timedelta, time, date, timezone
import pytz
from pydantic.v1 import BaseModel, Field, StrictStr, validator
from lusid.models.upsert_compliance_rule_request import UpsertComplianceRuleRequest
from lusid.models.reference_list_request import ReferenceListRequest
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from collections import defaultdict
from lusidjam import RefreshingToken

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/docs/how-do-i-use-an-api-access-token-with-the-lusid-sdk)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")
    print()

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook"),
    EnvironmentVariablesConfigurationLoader()]
api_factory = SyncApiClientFactory(config_loaders=config_loaders)
    
# Confirm success
api_client = api_factory.build(la.ApplicationMetadataApi)
api_url = api_client.api_client.configuration._base_path.replace("api","")
print ('LUSID Environment :', api_url + "docs")
display(pd.DataFrame(api_client.get_lusid_versions().to_dict()))

In [ ]:
# define some APIs
properties_api = api_factory.build(la.PropertyDefinitionsApi)
compliance_api = api_factory.build(la.ComplianceApi)
aggregation_api = api_factory.build(la.AggregationApi)
configuration_recipe_api = api_factory.build(la.ConfigurationRecipeApi)
portfolio_groups_api = api_factory.build(la.PortfolioGroupsApi)
portfolios_api = api_factory.build(la.PortfoliosApi)
instruments_api = api_factory.build(la.InstrumentsApi)
allocations_api = api_factory.build(lusid.api.AllocationsApi)
transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)
property_definitions_api = api_factory.build(la.PropertyDefinitionsApi)
quotes_api = api_factory.build(la.QuotesApi)
orders_api = api_factory.build(la.OrdersApi)

In [ ]:
# Specify a unique scope and code to segregate data in this tutorial from others
scope = "compliance_demo"
portfolio_code="US_Balanced"
portfolio_group_code="US_GROUP"
portfolio_base_currency="USD"
print(f"'{scope}/{portfolio_code}' scope and code created.")

In [ ]:
def parse_csv_date(date_str):
    return datetime.strptime(date_str, "%d/%m/%Y").replace(tzinfo=timezone.utc)

## Delete Existing Data

This section ensures a clean slate before new instruments and portfolios are created. It removes any pre-existing portfolios, portfolio groups, or instruments from the specified scope to prevent data collisions or duplication that may interfere with rule testing or portfolio construction.

**APIs & Methods Used:**
- `PortfolioGroupsApi.list_portfolio_groups(scope, filter)`
- `PortfolioGroupsApi.delete_portfolio_group(scope, code)`
- `TransactionPortfoliosApi.list_portfolios(scope)`
- `TransactionPortfoliosApi.delete_portfolio(scope, code)`
- `InstrumentsApi.delete_instruments(request_body)`
- `QuotesApi.list_quotes_for_scope(scope)`
- `QuotesApi.delete_quotes(scope, request_body)`
- `PropertyDefinitionsApi.list_property_definitions()`
- `PropertyDefinitionsApi.delete_property_definition(domain, scope, code)`
- `OrdersApi.list_orders()`
- `OrdersApi.delete_order(scope, code)`
- `ComplianceApi.list_compliance_templates()`
- `ComplianceApi.delete_compliance_template(scope, code)`
- `ComplianceApi.list_compliance_rules()`
- `ComplianceApi.delete_compliance_rules(scope, code)`

In [ ]:
delete_and_recreate_instruments = True #binary flag to remove and recreate all instruments in the specified scope

In [ ]:
# First determine what to delete
portfolio_groups = portfolio_groups_api.list_portfolio_groups(scope=scope, filter=f"id.code in ('{portfolio_group_code}')").values

portfolios = portfolios_api.list_portfolios_for_scope(scope=scope, filter=f"id.code in ('{portfolio_code}')").values

quotes = quotes_api.list_quotes_for_scope(scope = scope).values

if delete_and_recreate_instruments:
    instruments = [i.lusid_instrument_id for i in instruments_api.list_instruments(
        limit = 2000,
        filter="lusidInstrumentId startswith 'LU' and state eq 'Active'",
        scope = scope
    ).values]
else:
    instruments = []

property_definitions = property_definitions_api.list_property_definitions(filter=f"scope eq '{scope}'").values

all_orders = orders_api.list_orders()
orders_list = [
    order
    for order in all_orders.values
]
orders_df = lusid_response_to_data_frame(orders_list)
orders_df = orders_df[["id.scope", "id.code"]]
orders_to_delete = orders_df[orders_df['id.scope'] == scope]

all_compliance_templates = compliance_api.list_compliance_templates()
template_list = [
    template
    for template in all_compliance_templates.values
]
template_df = lusid_response_to_data_frame(template_list)
template_df = template_df[["id.scope", "id.code"]]
templates_to_delete = template_df[template_df['id.scope'] == scope]

all_compliance_rules = compliance_api.list_compliance_rules()
rules = [
    {"scope": rule.id.scope, "code": rule.id.code}
    for rule in all_compliance_rules.values
]
rules_df = pd.DataFrame(rules)
rules_to_delete = rules_df[rules_df["scope"] == scope]

print("The following will be DELETED:")
print(f"{len(portfolio_groups)} portfolio groups")
print(f"{len(portfolios)} portfolios")
print(f"{len(instruments)} Instruments")
print(f"{len(quotes)} Quotes")
print(f"{len(property_definitions)} property definitions")
print(f"{len(templates_to_delete)} compliance templates")
print(f"{len(rules_to_delete)} compliance rules")
print(f"{len(orders_to_delete)} orders")

In [ ]:
# Portfolio Groups
if len(portfolio_groups) > 0:
    
    for p in portfolio_groups:
        delete_response = portfolio_groups_api.delete_portfolio_group(scope = scope,code = p.id.code)
        if delete_response.staged_modifications is not None and delete_response.staged_modifications.count_pending > 0:
            raise SystemExit(f"Portfolio {scope}/{portfolio_code} cannot be deleted due to staging rules, please approve the delete and rerun the notebook")
    
    print(f"{len(portfolio_groups)} Portfolio Groups deleted")

# Portfolios
if len(portfolios) > 0:
    # Sort to have derived portfolios first
    portfolios.sort(key = lambda p: p.type)
    
    for p in portfolios:
        delete_response = portfolios_api.delete_portfolio(scope = scope,code = p.id.code)
        print
        if delete_response.staged_modifications.count_pending > 0:
            raise SystemExit(f"Portfolio {scope}/{portfolio_code} cannot be deleted due to staging rules, please approve the delete and rerun the notebook")
    
    print(f"{len(portfolios)} Portfolios deleted")

# Quotes - all the quotes in the scope
if len(quotes) > 0:
    delete_quotes_request = { f"{quote.quote_id.quote_series_id.instrument_id}_{quote.quote_id.effective_at}": { 
        "quoteSeriesId": quote.quote_id.quote_series_id, 
        "effectiveAt": quote.quote_id.effective_at
    } for quote in quotes }

    response = quotes_api.delete_quotes(scope = scope, request_body = delete_quotes_request)

    if len(response.failed) > 0:
        print("Unable to delete quotes - please investigate")
        
    print(f"{len(response.values)} quotes deleted")

# Instruments
if len(instruments) > 0:
    response = instruments_api.delete_instruments(scope=scope, request_body=instruments, delete_mode="Hard")
        
    print(f"{len(instruments)} instruments deleted")

# Property definitions
if(len(property_definitions) > 0):
    for p in property_definitions:
        response = property_definitions_api.delete_property_definition(domain = p.domain, scope = p.scope, code = p.code)

    print(f"{len(property_definitions)} Property Definitions deleted")

# Compliance Templates
if(len(templates_to_delete) > 0):
    for _, row in templates_to_delete.iterrows():
        template_code = row['id.code']
        compliance_api.delete_compliance_template(scope = scope, code = template_code)
    print(f"{len(templates_to_delete)} Compliance Templates deleted")

# Compliance Rules
if(len(rules_to_delete) > 0):
    for _, row in rules_to_delete.iterrows():
        rule_code = row['code']
        compliance_api.delete_compliance_rule(scope = scope, code = rule_code)
    print(f"{len(rules_to_delete)} Compliance Rules deleted")

# Orders
if(len(orders_to_delete) > 0):
    for _, row in orders_to_delete.iterrows():
        order_code = row['id.code']
        orders_api.delete_order(scope = scope, code = order_code)
    print(f"{len(orders_to_delete)} Orders deleted")

# Recipe
# Not currently being deleted to avoid creating slowness in LUSID

## Create Instruments, Portfolio, and Transactions

This section creates mock financial data for use in compliance testing including the following:
- `Instruments including stocks and bonds`
- `Instrument Properties representing classification schemes and ratings`
- `A "Recipe" that provides the rules that LUSID will use to value the portfolio`
- `Portfolios and a Portfolio Group`
- `Transactions that will determine the holdings within the Portfolio`
- `Market Data Quotes that will be used to value the Portfolio`

**APIs & Methods Used:**
- `InstrumentsApi.upsert_instruments(request)`
- `InstrumentsApi.upsert_instruments_properties(scope, request)`
- `TransactionPortfoliosApi.create_portfolio(request)`
- `TransactionPortfoliosApi.upsert_transactions(scope, code, request)`
- `PropertyDefinitionsApi.create_property_definition(request)`

### Instrument data

In [ ]:
df = pd.read_csv("data/demo-instrumentMaster.csv")
df

### Load instruments

In [ ]:
definitions = {}

for index, asset in df.iterrows():
    # Map identifier columns
    if pd.isnull(asset["Ticker"] ) or asset["Ticker"] == "":
        identifiers = {
            "Figi": lm.InstrumentIdValue(value = asset["FIGI"]),
            "Isin": lm.InstrumentIdValue(value = asset["ISIN"]),
        }
    else:
        identifiers = {
            "Figi": lm.InstrumentIdValue(value = asset["FIGI"]),
            "Isin": lm.InstrumentIdValue(value = asset["ISIN"]),
            "Ticker": lm.InstrumentIdValue(value = asset["Ticker"])  
        }

    # Equities
    if asset["Type"] in ["Equity", "ETF"]:
        definitions[asset["Company Name"]] = lm.InstrumentDefinition(
            name = asset["Company Name"],
            identifiers = identifiers,
            definition = lm.Equity(
                instrument_type = "Equity",
                dom_ccy = asset["currency"],
                identifiers = {}
            )
        )
    
    # Bonds (dates are guaranteed non-null now)
    elif asset["Type"] == "Bond":
        definitions[asset["Company Name"]] = lm.InstrumentDefinition(
            name = asset["Company Name"],
            identifiers = identifiers,
            definition = lm.Bond(
                instrument_type = "Bond",
                start_date = asset["Issue Date"],
                maturity_date = asset["Maturity Date"],
                dom_ccy = "USD",
                flow_conventions = lm.FlowConventions(
                    currency = asset["currency"],
                    payment_frequency = "6M",
                    day_count_convention = "ActualActual",
                    roll_convention = "NoAdjustment",
                    payment_calendars = [],
                    reset_calendars = [],
                    settle_days = 0,
                    reset_days = 0
                ),
                principal = 1,
                coupon_rate = float(asset["Coupon"])
            )
        )

# Upsert instruments to a custom scope in LUSID
upsert_instruments_response = instruments_api.upsert_instruments(
    request_body = definitions,
    scope = scope,
)

# Transform API response to a dataframe and show internally-generated unique LUID for each mastered instrument
upsert_instruments_response_df = lusid_response_to_data_frame(list(upsert_instruments_response.values.values()))
print(f"The following instruments have been added to the {scope} scope.")
display(upsert_instruments_response_df[["name", "lusidInstrumentId"]])

### Create properties

In [ ]:
property_definitions_dataset = pd.read_csv("data/demo-instrumentPropertiesDefinition.csv")
property_definitions_dataset

In [ ]:
for _, row in property_definitions_dataset.iterrows():
    request = lm.CreatePropertyDefinitionRequest(
        domain=row["domain"],
        scope = scope,
        code=row["code"],
        display_name=row["displayname"],
        data_type_id = lm.ResourceId(
            scope=row["data_type_id_scope"],
            code=row["data_type_id_code"]
        ),
        constraint_style=row["constraint_style"],
        lifetime=row["lifetime"],
        property_description=row["description"]
    )
    
    response = property_definitions_api.create_property_definition(create_property_definition_request = request)

    print(f"Successfully created {request.domain}/{request.scope}/{request.code}")

### Add Properties to Instruments

In [ ]:
df.rename(columns={'UltimateParent': 'ultimate_parent'}, inplace=True)

# Create a convenience function to call for each vendor dataframe
def add_properties_to_instruments(vendor_dataframe, property_scope, property_code):
    property_request = [
        lm.UpsertInstrumentPropertyRequest(
            identifier_type="Figi",
            identifier=security["FIGI"],
            properties = [
                lm.ModelProperty(
                    key=f"Instrument/{property_scope}/{property_code}",
                    value = lm.PropertyValue(
                        label_value = security[property_code]
                    )
                )
            ]
        )
        for index, security in vendor_dataframe.iterrows()
        if pd.notnull(security[property_code])  # This line filters out nulls
    ]

    upsert_properties_response = instruments_api.upsert_instruments_properties(
        scope=f"{property_scope}",
        upsert_instrument_property_request = property_request,
    )
    
    print(f"{property_code} properties from {property_scope} updated at {(str(upsert_properties_response.as_at_date))}")
    
    
add_properties_to_instruments(df, scope, "custom_asset_class")
add_properties_to_instruments(df, scope, "custom_sector")
add_properties_to_instruments(df[df["Type"]=="Bond"], scope, "rating_sp")
add_properties_to_instruments(df[df["Type"]=="Bond"], scope, "rating_moodys")
add_properties_to_instruments(df, scope, "ultimate_parent")

In [ ]:
# Add Property values to a Cash position
USD_CCY_property_request = [
    lm.UpsertInstrumentPropertyRequest(
        identifier_type="LusidInstrumentId",
        identifier="CCY_USD",
        properties = [
            lm.ModelProperty(
                key=f"Instrument/{scope}/custom_asset_class",
                value = lm.PropertyValue(
                    label_value="Cash"
                    )
                )
            ]
        ),
    lm.UpsertInstrumentPropertyRequest(
        identifier_type="LusidInstrumentId",
        identifier="CCY_USD",
        properties = [
            lm.ModelProperty(
                key=f"Instrument/{scope}/custom_sector",
                value = lm.PropertyValue(
                    label_value="Cash"
                    )
                )
            ]
        ),
    lm.UpsertInstrumentPropertyRequest(
        identifier_type="LusidInstrumentId",
        identifier="CCY_USD",
        properties = [
            lm.ModelProperty(
                key=f"Instrument/{scope}/ultimate_parent",
                value = lm.PropertyValue(
                    label_value="USD_Cash"
                    )
                )
            ]
        ),
    ]

upsert_properties_response = instruments_api.upsert_instruments_properties(
        scope="default",
        upsert_instrument_property_request = USD_CCY_property_request,
    )

### Create recipes

In [ ]:
recipe_code = "demo_valuation_recipe_mid"

# Create a recipe to perform a valuation
configuration_recipe = lm.ConfigurationRecipe(
    scope = scope,
    code = recipe_code,
    market = lm.MarketContext(
        market_rules = [
            # define how to resolve the quotes
            lm.MarketDataKeyRule(
                key="Quote.Isin.*",
                supplier="Lusid",
                data_scope = scope,
                quote_type="Price",
                field="mid",
            ),
        ],
        options = lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="Isin",
            default_scope = scope,
        ),
    ),
    pricing = lm.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": True},
        model_rules= [
            lm.VendorModelRule(
                    supplier="Lusid",
                    modelName="BondLookupPricer",
                    instrumentType="Bond",
                    parameters="",
                    modelOptions= lm.EmptyModelOptions(
                        model_options_type="EmptyModelOptions"
                    ),
            ),
        ],
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request = lm.UpsertRecipeRequest(
            configuration_recipe = configuration_recipe
        )
    )
)

### Create portfolio

In [ ]:
# Create the request to add portfolio to LUSID
transaction_portfolio_request = lm.CreateTransactionPortfolioRequest(
    display_name="US Balanced Fund",
    code = portfolio_code,
    base_currency="USD",
    created = datetime(2024, 1, 1, tzinfo = timezone.utc),
#    sub_holding_keys=[
#        strategy_property_code,
#        substrategy_property_code
#    ],
#    transaction_type_scope = transaction_type_scope,
    instrument_scopes = [ scope ],
    amortisation_method="StraightLine",
    instrument_event_configuration = lm.InstrumentEventConfiguration(
        recipe_id = lm.ResourceId(
            scope = scope,
            code = recipe_code
        )
    )
)

# Call LUSID to create the portfolio
response = transaction_portfolios_api.create_portfolio(
    scope = scope,
    create_transaction_portfolio_request = transaction_portfolio_request,
)

print(f"Portfolio {scope}/{portfolio_code} created")

### Create portfolio group

In [ ]:
def create_portfolio_group(portfolio_groups_api, scope, code, portfolios):

    portfolio_creation_date = datetime(2020, 1, 1, tzinfo = pytz.utc)

    try:
        portfolio_groups_api.delete_portfolio_group(scope = scope, code = code)
    except:
        pass

    group_request = lm.CreatePortfolioGroupRequest(
        code = code,
        display_name = code,
        values = portfolios,
        sub_groups = None,
        description = None,
        created = portfolio_creation_date,
    )

    portfolio_group = portfolio_groups_api.create_portfolio_group(
        scope = scope, create_portfolio_group_request = group_request
    )

    return portfolio_group

In [ ]:
result = create_portfolio_group(portfolio_groups_api, scope, portfolio_group_code, [lm.ResourceId(scope = scope, code = portfolio_code)])

### Load transactions

In [ ]:
txn_df = pd.read_csv("data/demo-transactions.csv")
txn_df

In [ ]:
transactions_request = []

for index, txn in txn_df.iterrows():
    # Append your request to the list
    transactions_request.append(lm.TransactionRequest(
        transaction_id=txn["txn_id"],
        type=txn["txn_type"],
        instrument_identifiers={
            "Instrument/default/Isin": txn["ISIN"],
            "Instrument/default/Figi": txn["FIGI"],
        },
        transaction_date=parse_csv_date(txn["txn_date"]).isoformat(),
        settlement_date=parse_csv_date(txn["txn_settle_date"]).isoformat(),
        units=txn["units"],
        transaction_price=lm.TransactionPrice(
              price=txn["Price"],
              type="Price"),
        total_consideration=lm.CurrencyAndAmount(
          amount=txn["total"],
          currency=txn["currency"]),
    ))
    
txn_response = transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=transactions_request
)

print("Transactions upserted")

### Load quotes

In [ ]:
quotes_df = pd.read_csv("data/demo-quotes.csv")
# Compliance runs at latest asat
quotes_df["date"] = quotes_df["date"].apply(
    lambda s: datetime.now(pytz.UTC).strftime("%d-%b-%y")
)
quotes_df

In [ ]:
# Initialise an empty instrument quotes list to hold the quotes
quotes_request = {}

# Iterate over the quotes
for index, quote in quotes_df.iterrows():
    quote_date = pd.to_datetime(quote["date"]).strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    if not pd.isna(quote["scale_factor"]):
        scale_factor = quote["scale_factor"]
    else:
        scale_factor = None    

    quotes_request[str(index)] = lm.UpsertQuoteRequest(
        quote_id = lm.QuoteId(
            quote_series_id = lm.QuoteSeriesId(
                provider="Lusid",
                instrument_id=quote["ISIN"],
                instrument_id_type="Isin",
                quote_type="Price",
                field="mid",
            ),
            effective_at = quote_date,
        ),
        metric_value = lm.MetricValue(
            value=quote["price"],
            unit="USD",
        ),
        scale_factor=scale_factor,
        lineage="InternalSystem",
    )
    
# Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(scope = scope, request_body = quotes_request)

response.failed

### Run a test valuation

In [ ]:
def generate_valuation_request(valuation_effectiveAt):

    # Create the valuation request
    valuation_request = lm.ValuationRequest(
        recipe_id = lm.ResourceId(
            scope=scope, code="demo_valuation_recipe_mid"
        ),
        metrics = [
            lm.AggregateSpec(key = "Instrument/default/Name", op = "Value"),
            lm.AggregateSpec(key = "Valuation/PvInReportCcy", op = "Proportion"),
            lm.AggregateSpec(key = "Valuation/PvInReportCcy", op = "Sum"),
            lm.AggregateSpec(key = "Holding/default/Units", op = "Sum"),
            lm.AggregateSpec(key = "Aggregation/Errors", op = "Value"),
        ],
        group_by=["Instrument/default/Name"],
        #filters=[lm.PropertyFilter('Instrument/default/Name', 'NotEquals', 'USD'),
        #        lm.PropertyFilter('Instrument/default/Name', 'NotEquals', 'USD Cash')
        #],
        portfolio_entity_ids = [
            lm.PortfolioEntityId(scope = scope, code = portfolio_code)
        ],
        valuation_schedule = lm.ValuationSchedule(
            effective_at = valuation_effectiveAt.isoformat()
        ),
    )

    return valuation_request

In [ ]:
# we will use the mid price
aggregation = aggregation_api.get_valuation(
    valuation_request = generate_valuation_request(datetime.now(pytz.UTC))
)
pd.DataFrame(aggregation.data)

### Create Derived Properties

This section defines **derived properties**—expressions that are calculated dynamically based on instrument or portfolio properties, allowing compliance rules to reference computed values.  and promotes reuse of logical components across multiple templates or rules.  In the below example, we will be creating properties to convert bond letter ratings to a numerical value, determine the best rating across multiple values and allow for the calculation of an average rating for a portfolio. Additionally, we will create a derived property that will enable us to calculate the weighted average duration for a portfolio that contains fixed income positions.

**APIs & Methods Used:**
- `DerivedPropertyDefinitionApi.upsert_derived_property_definition(request)`Derived properties

In [ ]:
# "Convert S&P rating to number" property 
request1 = lm.CreateDerivedPropertyDefinitionRequest(
    domain="Instrument",
    scope = scope,
    code="SP_numerical_rating",
    display_name="S&P Rating - Numerical Equivalent",
    data_type_id = lm.ResourceId(
        scope="system",
        code="number"
    ),
    lifetime="Perpetual",
    property_description="Conversion of S&P letter ratings to numerical value for the purpose of calculating a portfolio average",
    derivation_formula=f"map(Properties[Instrument/{scope}/rating_sp]: 'AAA'=1, 'AA+'=2, 'AA'=3, 'AA-'=4, 'A+'=5, 'A'=6, 'A-'=7, 'BBB+'=8, 'BBB'=9, 'BBB-'=10, 'BB+'=11, 'BB'=12, 'BB-'=13)",
    is_filterable = False
)
    
response1 = property_definitions_api.create_derived_property_definition(create_derived_property_definition_request = request1)

print(f"Successfully created {request1.domain}/{request1.scope}/{request1.code}")

# "Convert Moodys rating to number" property 
request2 = lm.CreateDerivedPropertyDefinitionRequest(
    domain="Instrument",
    scope = scope,
    code="Moodys_numerical_rating",
    display_name="Moodys Rating - Numerical Equivalent",
    data_type_id = lm.ResourceId(
        scope="system",
        code="number"
    ),
    lifetime="Perpetual",
    property_description="Conversion of S&P letter ratings to numerical value for the purpose of calculating a portfolio average",
    derivation_formula=f"map(Properties[Instrument/{scope}/rating_moodys]: 'Aaa'=1, 'Aa1'=2, 'Aa2'=3, 'Aa3'=4, 'A1'=5, 'A2'=6, 'A3'=7, 'Baa1'=8, 'Baa2'=9, 'Baa3'=10, 'Ba1'=11, 'Ba2'=12, 'Ba'=12,'Ba3'=13)",
    is_filterable = False
)
    
response2 = property_definitions_api.create_derived_property_definition(create_derived_property_definition_request = request2)

print(f"Successfully created {request2.domain}/{request2.scope}/{request2.code}")

In [ ]:
# "Best Rating" property 
request3 = lm.CreateDerivedPropertyDefinitionRequest(
    domain="Instrument",
    scope = scope,
    code="numerical_credit_rating",
    display_name="Credit Rating - Numerical",
    data_type_id = lm.ResourceId(
        scope="system",
        code="number"
    ),
    lifetime="Perpetual",
    property_description="Lower numerical credit rating between S&P and Moody's",
    derivation_formula=f"MAX (Properties[Instrument/{scope}/SP_numerical_rating] , Properties[Instrument/{scope}/Moodys_numerical_rating])",
    is_filterable = False
)
    
response3 = property_definitions_api.create_derived_property_definition(create_derived_property_definition_request = request3)

print(f"Successfully created {request3.domain}/{request3.scope}/{request3.code}")

In [ ]:
# Rating x PV Property
request4 = lm.CreateDerivedPropertyDefinitionRequest(
    domain="DerivedValuation",
    scope = scope,
    code="pv_x_numerical_credit_rating",
    display_name="PV x Credit Rating - Numerical",
    success_mode="Atomic",
    data_type_id = lm.ResourceId(
        scope="system",
        code="number"
    ),
    lifetime="TimeVariant",
    property_description="Holding PV x Numerical Credit Rating",
    derivation_formula=f"AddressKeys[Valuation/PvInPortfolioCcy] * Properties[Instrument/{scope}/numerical_credit_rating]",
    is_filterable = False
)
    
response4 = property_definitions_api.create_derived_property_definition(create_derived_property_definition_request = request4)

print(f"Successfully created {request4.domain}/{request4.scope}/{request4.code}")

In [ ]:
# Duration x PV Property
request5 = lm.CreateDerivedPropertyDefinitionRequest(
    domain="DerivedValuation",
    scope = scope,
    code="pv_x_modified_duration",
    display_name="PV x Modified Duration",
    success_mode="Atomic",
    data_type_id = lm.ResourceId(
        scope="system",
        code="number"
    ),
    lifetime="TimeVariant",
    property_description="Holding PV x Numerical Credit Rating",
    derivation_formula=f"AddressKeys[Valuation/PvInPortfolioCcy] * AddressKeys[Analytic/ModifiedDuration]",
    is_filterable = False
)
  
response5 = property_definitions_api.create_derived_property_definition(create_derived_property_definition_request = request5)

print(f"Successfully created {request5.domain}/{request5.scope}/{request5.code}")

## Create Compliance Templates and Rules

This section creates custom compliance templates that define the logical structure of a compliance rule (e.g., group by sector, apply filter, evaluate condition) and custom compliance rules that are specific applications of a template, tied to derived or raw properties, with set parameters. They define compliance logic modularly and consistently across portfolios, allowing compliance officers to evaluate portfolio compliance based on firm policies or regulatory requirements.

**APIs & Methods Used:**
- `ComplianceRuleTemplatesApi.upsert_compliance_rule_template(request)`
- `ComplianceRulesApi.upsert_compliance_rule(request)`

In [ ]:
# Import Custom Template Step Requests
from lusid.models.check_step_request import CheckStepRequest
from lusid.models.percent_check_step_request import PercentCheckStepRequest
from lusid.models.filter_step_request import FilterStepRequest
from lusid.models.group_by_step_request import GroupByStepRequest
from lusid.models.group_filter_step_request import GroupFilterStepRequest
from lusid.models.create_compliance_template_request import CreateComplianceTemplateRequest
from lusid.models.compliance_template_variation_request import ComplianceTemplateVariationRequest

### % Allocation Rule

##### Check Allocation %Thresholds - Create Custom Compliance Template

This section demonstrates how to implement a custom compliance rule in LUSID to monitor allocations to a single issuer. Specifically, we are setting up a rule that ensures that no single issuer exposure, defined by our "ultimate parent" property created above does not exceed 10% of the portfolio's total value. A warning is also triggered if the exposure goes above 8%.

In [ ]:
##### Define the steps that make up the compliance template

# These steps define the logic that will be reused in rules.
steps = [
    # Group the portfolio data by portfolio ID
    GroupByStepRequest(complianceStepTypeRequest= "GroupByStepRequest", label= "PortfolioIdGroup"),
    
    # Then group by the classification of the asset (e.g., ultimate parent)
    GroupByStepRequest(complianceStepTypeRequest= "GroupByStepRequest", label= "ClassificationGroup"),
    
    # Finally, check whether the percentage of this classification is within bounds
    PercentCheckStepRequest(complianceStepTypeRequest= "PercentCheckStepRequest", label= "Compare"),    
]

##### Define the compliance template variation

# A variation is a named instance of a compliance template, used when defining rules.
variation = ComplianceTemplateVariationRequest(
    label = f"{scope}-issuer_threshold-percent",
    description = "Check to ensure that no single issuer exposure exceeds 10% of the portfolio",
    outcome_description="Single-issuer exposure exceeds 10% ",
    steps = steps
)

##### Create the compliance template

# This template acts like a reusable rule blueprint.
create_compliance_template_request = CreateComplianceTemplateRequest(
    code=f'{scope}-classification_threshold-percent',
    description="Custom compliance template to check grouping thresholds do not exceed limits",
    variations = [ variation ]
)

# Attempt to create the template. Catch the exception if it already exists.
try:
    compliance_api.create_compliance_template(scope = scope, 
    create_compliance_template_request = create_compliance_template_request
    )
except Exception as e:
    error_message = str(e)
    if "EntityWithIdAlreadyExists" in error_message:
        print("Caught an ApiException for entity already exists")
    else:
        raise

##### Check Allocation %Thresholds - Create Custom Compliance Rule

Create a custom rule with the specific parameters checking whether the portfolio exceeds the 10% limit for "Ultimate Parent" and displays a warning if the allocation is more than 8%.

In [ ]:
##### Define the Custom Compliance Rule

# Now that we have a reusable template, we define a rule that enforces it.

# Portfolio grouping key to apply the rule per portfolio
portfolio_grouping_key = "Properties[Portfolio/default/Id]"

# Define the grouping by Ultimate Parent
sector_grouping_key = f"Properties[Instrument/{scope}/ultimate_parent]"

# Create the rule that uses the above template and applies it to your portfolios.
upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code='single_issuer_10pct_max'),  # Unique identifier for the rule
    name="Single Issuer Exposure cannot exceed 10%",
    description="Single Issuer Weight cannot exceed 10%",

    # Attach the previously defined template and variation
    template_id=lm.ResourceId(scope=scope, code=f"{scope}-classification_threshold-percent"),
    variation=f"{scope}-issuer_threshold-percent",
    portfolio_group_id = lm.ResourceId(scope = scope, code = portfolio_group_code),

    active = True,  # Activate the rule

    # Define the parameters required by the steps in the template
    parameters = {
        # Step 1: Group by Portfolio ID
        "PortfolioIdGroup.GroupingKey": lm.GroupBySelectorComplianceParameter(
            value = portfolio_grouping_key,
            compliance_parameter_type='GroupBySelectorComplianceParameter'),

        # Step 2: Group by Ultimate Parent
        "ClassificationGroup.GroupingKey": lm.GroupBySelectorComplianceParameter(
            value = sector_grouping_key,
            compliance_parameter_type='GroupBySelectorComplianceParameter'),

        # Step 3a: Denominator for percentage calculation - total portfolio value
        "Compare.Denominator": lm.GroupCalculationComplianceParameter(
            value="Sum(PreviousStep[PortfolioIdGroup].Results[Valuation/PvInPortfolioCcy])",
            compliance_parameter_type="GroupCalculationComplianceParameter"),

        # Step 3b: Numerator - total value of international equities
        "Compare.Numerator": lm.GroupCalculationComplianceParameter(
            value="Sum(Results[Valuation/PvInPortfolioCcy])",
            compliance_parameter_type="GroupCalculationComplianceParameter"),

        # Step 4a: Define Hard boundaries: > 10% will fail the rule
        "Compare.UpperBound": lm.DecimalComplianceParameter(
            value = 10.0,
            compliance_parameter_type="DecimalComplianceParameter"),
        "Compare.LowerBound": lm.DecimalComplianceParameter(
            value = -1.0,
            compliance_parameter_type="DecimalComplianceParameter"),

        # Step 4b: DefineWarning thresholds: > 8% will raise a warning
        "Compare.UpperWarning": lm.DecimalComplianceParameter(
            value = 8.0,
            compliance_parameter_type="DecimalComplianceParameter"),
        "Compare.LowerWarning": lm.DecimalComplianceParameter(
            value = -1.0,
            compliance_parameter_type="DecimalComplianceParameter"),
    },

    properties = {}  # Optional metadata
)

##### Submit the compliance rule to LUSID
# This makes the rule live and available to run in compliance checks.
compliance_api.upsert_compliance_rule(upsert_compliance_rule_request = upsert_compliance_rule_request)

print("Rule created")

### Restricted Investments Rule

##### Restricted Investments - Create Custom Compliance Template

This section demonstrates how to implement a custom compliance rule in LUSID to monitor restricted investments (e.g. based on specific instrument IDs or instrument property values). Specifically, we are setting up a rule that ensures that all investments are US Dollar denominated.

In [ ]:
##### Define Steps for the Compliance Template

# The template will use 3 logical steps:
# 1. Group portfolio data by portfolio ID.
# 2. Exclude Cash from portfolios.
# 2. Group by holding/instrument
# 3. Evaluate a custom condition (e.g., is the instrument USD denominated).

steps = [
    GroupByStepRequest(complianceStepTypeRequest= "GroupByStepRequest", label= "PortfolioIdGroup"),
    FilterStepRequest(complianceStepTypeRequest= "FilterStepRequest", label= "ExcludeCashFilter"),
    GroupByStepRequest(complianceStepTypeRequest= "GroupByStepRequest", label= "HoldingGroup"),
    GroupByStepRequest(complianceStepTypeRequest= "GroupByStepRequest", label= "IsinGroup"), # these additional groupings ensure 
    GroupByStepRequest(complianceStepTypeRequest= "GroupByStepRequest", label= "NameGroup"), # more descriptive detail in the rule output
    CheckStepRequest(complianceStepTypeRequest= "CheckStepRequest", label= "Compare"),    
]

##### Define Template Variation
variation = ComplianceTemplateVariationRequest(
    label = f"{scope}-instrument_check",
    description = "Check to ensure that all instruments meet specific criteria",
    outcome_description="Instrument is non-USD denominated",
    steps = steps
)

##### Create the Compliance Template
create_compliance_template_request = CreateComplianceTemplateRequest(
    code=f'{scope}-instrument_criteria_check',
    description='Custom compliance template checking that instruments meeting specified criteria',
    variations = [ variation ]
)

# If the template already exists, catch and ignore the exception.
try:
    compliance_api.create_compliance_template(
        scope = scope, 
        create_compliance_template_request = create_compliance_template_request
    )
except Exception as e:
    error_message = str(e)
    if "EntityWithIdAlreadyExists" in error_message:
        print("Caught an ApiException for entity already exists")
    else:
        raise

##### Restricted Investments - Create Custom Compliance Rule

Create the custom rule that ensures that all investments are US Dollar denominated.

In [ ]:
##### Define Rule Parameters

# These values specify the logic for filtering and rating thresholds.

currency_check = 'USD'
portfolio_grouping_key = "Properties[Portfolio/default/Id]"

# Define grouping for instruments
holding_grouping_key = "HoldingId"
isin_grouping_key = "Properties[Instrument/default/Isin]"
name_grouping_key = "Properties[Instrument/default/Name]"

# Define filter and check criteria
filter_predicate = "instrumentType not in ('Cash', 'UnsettledCash')"
compare_predicate = f"Properties[Holding/default/Currency].Singular eq '{currency_check}'"
#compare_predicate = f"Properties[Analytic/default/DomCcy].Singular eq '{currency_check}'"

##### Define the Compliance Rule

# This rule applies the above logic using the template and parameters.

upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code='usd_denominated_only'),
    name='All USD Securities Held',
    description='Check to ensure that all securities held in the portfolio are USD denominated',
    
    # Connect to the previously defined template and variation
    template_id=lm.ResourceId(scope=scope, code=f'{scope}-instrument_criteria_check'),
    variation=f"{scope}-instrument_check",
    
    portfolio_group_id = lm.ResourceId(scope = scope, code = portfolio_group_code),
    active = True,

    # Define values for each step in the template
    parameters = {
        "PortfolioIdGroup.GroupingKey": lm.GroupBySelectorComplianceParameter(
            value= portfolio_grouping_key,
            compliance_parameter_type='GroupBySelectorComplianceParameter'),

        "ExcludeCashFilter.Predicate": lm.FilterPredicateComplianceParameter(
            value= filter_predicate,
            compliance_parameter_type='FilterPredicateComplianceParameter'),

        "HoldingGroup.GroupingKey": lm.FilterPredicateComplianceParameter(
            value= holding_grouping_key,
            compliance_parameter_type='GroupBySelectorComplianceParameter'),

        "IsinGroup.GroupingKey": lm.FilterPredicateComplianceParameter(
            value= isin_grouping_key,
            compliance_parameter_type='GroupBySelectorComplianceParameter'),

        "NameGroup.GroupingKey": lm.FilterPredicateComplianceParameter(
            value= name_grouping_key,
            compliance_parameter_type='GroupBySelectorComplianceParameter'),

        "Compare.HardPredicate": lm.GroupFilterPredicateComplianceParameter(
            value= compare_predicate,
            compliance_parameter_type='GroupFilterPredicateComplianceParameter'),

        "Compare.SoftPredicate": lm.GroupFilterPredicateComplianceParameter(
            value= compare_predicate,
            compliance_parameter_type='GroupFilterPredicateComplianceParameter')
    },
    
    properties = {}  # Optional metadata
)

##### Submit the Rule to LUSID
compliance_api.upsert_compliance_rule(upsert_compliance_rule_request = upsert_compliance_rule_request)

print("Rule created")

### Maximum Exposure to Credit Rating Check

##### Check Exposure to High Yield Ratings - Create Custom Compliance Template

This example shows how to define a compliance rule to monitor the exposure to specific property values. The rule will filter the portfolio to use only the bond holdings and flag if the portfolio exposure to instruments with less than a BB+ credit rating is greater than 3%.

In [ ]:
##### Define Steps for the Compliance Template

# The template will use 3 logical steps:
# 1. Group portfolio data by portfolio ID.
# 2. Group by instrument
# 3. Filter by instrument and a minimum credit rating
# 3. Evaluate a custom condition (e.g., is there more than 3% allocated to ratings below BB+).

steps = [
    GroupByStepRequest(complianceStepTypeRequest= "GroupByStepRequest", label= "PortfolioIdGroup"),
    GroupByStepRequest(complianceStepTypeRequest= "GroupByStepRequest", label= "InstrumentTypeGroup"),
    FilterStepRequest(complianceStepTypeRequest= "FilterStepRequest", label= "InstrumentTypeFilter"),
    FilterStepRequest(complianceStepTypeRequest= "FilterStepRequest", label= "CreditRatingFilter"),
    CheckStepRequest(complianceStepTypeRequest= "CheckStepRequest", label= "Compare"),    
]

##### Define Template Variation
variation = ComplianceTemplateVariationRequest(
    label = f"{scope}-BB_rating_3pct_threshold",
    description = "Check to ensure no more than 3% of bond holdings are less than BB+ rated",
    outcome_description="Maximum amount of High Yield bonds exceeded",
    steps = steps
)

##### Create the Compliance Template
create_compliance_template_request = CreateComplianceTemplateRequest(
    code=f'{scope}-high_yield_credit_check',
    description='Custom compliance template checking that instruments meeting specified criteria',
    variations = [ variation ]
)

# If the template already exists, catch and ignore the exception.
try:
    compliance_api.create_compliance_template(
        scope = scope, 
        create_compliance_template_request = create_compliance_template_request
    )
except Exception as e:
    error_message = str(e)
    if "EntityWithIdAlreadyExists" in error_message:
        print("Caught an ApiException for entity already exists")
    else:
        raise

##### Check Exposure to High Yield Ratings - Create Custom Compliance Rule

Create the compliance rule to ensure that the portfolio exposure to instruments with less than a BB+ credit rating is not greater than 3%.

In [ ]:
##### Define Rule Parameters
portfolio_grouping_key = "Properties[Portfolio/default/Id]"
instrument_type_grouping_key = "instrumentType"

# Define grouping and filtering for the asset class of interest (international equities)
instrument_type_filter_predicate = "instrumentType IN ('Bond', 'ComplexBond')"
credit_rating_filter_predicate = f"Properties[Instrument/{scope}/numerical_credit_rating] gte 12.0"
compare_hard_predicate = "Sum(Results[Valuation/PvInPortfolioCcy])/Sum(PreviousStep[PortfolioIdGroup].Results[Valuation/PvInPortfolioCcy])*100 lt 3.0" 

##### Define the Compliance Rule

# This rule applies the above logic using the template and parameters.

upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code='minimum_BB_rated_limit'),
    name='No more than 3% of bond holdings less than BB+ rated',
    description='Check to ensure that no more than 3% of bond holdings have less than BB+ rating',
    
    # Connect to the previously defined template and variation
    template_id=lm.ResourceId(scope=scope, code=f'{scope}-high_yield_credit_check'),
    variation=f"{scope}-BB_rating_3pct_threshold",
    
    portfolio_group_id = lm.ResourceId(scope = scope, code = portfolio_group_code),
    active = True,

    # Define values for each step in the template
    parameters = {
        "PortfolioIdGroup.GroupingKey": lm.GroupBySelectorComplianceParameter(
            value= portfolio_grouping_key,
            compliance_parameter_type='GroupBySelectorComplianceParameter'),

        "InstrumentTypeGroup.GroupingKey": lm.FilterPredicateComplianceParameter(
            value= instrument_type_grouping_key,
            compliance_parameter_type='GroupBySelectorComplianceParameter'),

        "InstrumentTypeFilter.Predicate": lm.FilterPredicateComplianceParameter(
            value= instrument_type_filter_predicate,
            compliance_parameter_type='FilterPredicateComplianceParameter'),

        "CreditRatingFilter.Predicate": lm.FilterPredicateComplianceParameter(
            value= credit_rating_filter_predicate,
            compliance_parameter_type='FilterPredicateComplianceParameter'),

        "Compare.HardPredicate": lm.GroupFilterPredicateComplianceParameter(
            value= compare_hard_predicate ,
            compliance_parameter_type='GroupFilterPredicateComplianceParameter'),

        "Compare.SoftPredicate": lm.GroupFilterPredicateComplianceParameter(
            value= "1 eq 1", # this will result in the rule effectively being pass/fail with no warnings
            compliance_parameter_type='GroupFilterPredicateComplianceParameter')
    },
  
    properties = {}  # Optional metadata
)

##### Submit the Rule to LUSID
compliance_api.upsert_compliance_rule(upsert_compliance_rule_request = upsert_compliance_rule_request)

print("Rule created")

### Portfolio Weighted Average Credit Rating Check

##### Check Average Credit Rating - Create Custom Compliance Template

This example shows how to define a compliance rule to monitor the average credit rating of bond holdings. The rule will filter the portfolio to use only the bond holdings and flag if the average rating falls below A, and raise a warning if it drops below A+.

In [ ]:
##### Define Steps for the Compliance Template

# The template will use 3 logical steps:
# 1. Group portfolio data by portfolio ID.
# 2. Filter for specific instrument types (i.e., bonds).
# 3. Evaluate a custom condition (e.g., average credit rating check).

steps = [
    GroupByStepRequest(complianceStepTypeRequest= "GroupByStepRequest", label= "PortfolioIdGroup"),
    FilterStepRequest(complianceStepTypeRequest= "FilterStepRequest", label= "InstrumentTypeFilter"),
    CheckStepRequest(complianceStepTypeRequest= "CheckStepRequest", label= "Compare"),    
]

##### Define Template Variation
variation = ComplianceTemplateVariationRequest(
    label = f"{scope}-average_credit_rating_check",
    description = "Check to ensure that the average credit rating of all bond holdings meets the minimum threshold",
    outcome_description="The average credit rating of bond holdings is below the minimum rating",
    steps = steps
)

##### Create the Compliance Template
create_compliance_template_request = CreateComplianceTemplateRequest(
    code=f'{scope}-average_credit_rating',
    description='Custom compliance template checking classification thresholds do not exceed limits',
    variations = [ variation ]
)

# If the template already exists, catch and ignore the exception.
try:
    compliance_api.create_compliance_template(
        scope = scope, 
        create_compliance_template_request = create_compliance_template_request
    )
except Exception as e:
    error_message = str(e)
    if "EntityWithIdAlreadyExists" in error_message:
        print("Caught an ApiException for entity already exists")
    else:
        raise

##### Check Average Credit Rating - Create Custom Compliance Rule

Creates a custom rule that checks if the average bond rating for the portfolio is A or better and generates a warning if the rating is less than A. Our custom property established that the numerical equivalent of an A rating is 6. So any average rating less than 6 is within compliance. We will set a 'warning' if the average rating is less than A+ which would correspond to an average numerical rating greater than 5.

In [ ]:
##### Define Rule Parameters
credit_limit_threshold = 6         # A = 6, so any average above is a violation based on how the derived property was defined
credit_warning_threshold = 5       # A+ = 5, so anything above is a warning
portfolio_grouping_key = "Properties[Portfolio/default/Id]"

# Filter to include only bonds
sector_filter_predicate = "instrumentType IN ('Bond', 'ComplexBond')"

# Hard check (compliance violation) if average > 6
compare_hard_predicate = f"(Sum(Results[DerivedValuation/{scope}/pv_x_numerical_credit_rating])/Sum(Results[Valuation/PvInPortfolioCcy])) lte {credit_limit_threshold}"

# Soft check (warning) if average > 5
compare_soft_predicate = f"(Sum(Results[DerivedValuation/{scope}/pv_x_numerical_credit_rating])/Sum(Results[Valuation/PvInPortfolioCcy])) lte {credit_warning_threshold}"

##### Define the Compliance Rule
# This rule applies the above logic using the template and parameters.

upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code='portfolio_credit_rating_gte_A'),
    name='Portfolio Average Credit Rating A-Rated or above',
    description='Check to ensure that the average rating of bond holdings within a portfolio is A or better',
    
    # Connect to the previously defined template and variation
    template_id=lm.ResourceId(scope=scope, code=f'{scope}-average_credit_rating'),
    variation=f"{scope}-average_credit_rating_check",
    
    portfolio_group_id = lm.ResourceId(scope = scope, code = portfolio_group_code),
    active = True,

    # Define values for each step in the template
    parameters = {
        "PortfolioIdGroup.GroupingKey": lm.GroupBySelectorComplianceParameter(
            value= portfolio_grouping_key,
            compliance_parameter_type='GroupBySelectorComplianceParameter'),

        "InstrumentTypeFilter.Predicate": lm.FilterPredicateComplianceParameter(
            value= sector_filter_predicate,
            compliance_parameter_type='FilterPredicateComplianceParameter'),

        "Compare.HardPredicate": lm.GroupFilterPredicateComplianceParameter(
            value= compare_hard_predicate,
            compliance_parameter_type='GroupFilterPredicateComplianceParameter'),

        "Compare.SoftPredicate": lm.GroupFilterPredicateComplianceParameter(
            value= compare_soft_predicate,
            compliance_parameter_type='GroupFilterPredicateComplianceParameter')
    },
    
    properties = {}  # Optional metadata
)

##### Submit the Rule to LUSID
compliance_api.upsert_compliance_rule(upsert_compliance_rule_request = upsert_compliance_rule_request)

print("Rule created")

### Portfolio Weighted Average Duration Check

##### Check Portfolio Weighted Average Duration - Create Custom Compliance Rule

Creates a custom rule that checks if the weighted average portfolio duration is greater than a specified level. This template will need to filter by portfolio and by asset class and then utilize a custom derived property to calculate the weighted average duration.

In [ ]:
##### Define Steps for the Compliance Template

steps = [
    FilterStepRequest(complianceStepTypeRequest= "FilterStepRequest", label= "InstrumentTypeFilter"),
    GroupByStepRequest(complianceStepTypeRequest= "GroupByStepRequest", label= "PortfolioIdGroup"),
    CheckStepRequest(complianceStepTypeRequest= "CheckStepRequest", label= "Compare"),    
]

##### Define Template Variation
variation = ComplianceTemplateVariationRequest(
    label = f"{scope}-portfolio_duration_check",
    description = "Check the weighted average portfolio duration",
    outcome_description="The portfolio duration exceeds the limit",
    steps = steps
)

##### Create the Compliance Template
create_compliance_template_request = CreateComplianceTemplateRequest(
    code=f'{scope}-portfolio_duration_check',
    description='Custom compliance template checking the weighted average portfolio duration',
    variations = [ variation ]
)

# If the template already exists, catch and ignore the exception.
try:
    compliance_api.create_compliance_template(
        scope = scope, 
        create_compliance_template_request = create_compliance_template_request
    )
except Exception as e:
    error_message = str(e)
    if "EntityWithIdAlreadyExists" in error_message:
        print("Caught an ApiException for entity already exists")
    else:
        raise

##### Check Portfolio Weighted Average Duration - Create Custom Compliance Rule

Creates a custom rule that checks if the weighted average portfolio duration is greater than a specified level. This template will need to filter by portfolio and by asset class and then utilize a custom derived property to calculate the weighted average duration.

In [ ]:
##### Define Rule Parameters

# These values specify the logic for filtering and rating thresholds.

duration_limit_threshold = 8         
duration_warning_threshold = 7       
portfolio_grouping_key = "Properties[Portfolio/default/Id]"

# Filter to include only bonds
instrument_type_filter_predicate = "instrumentType IN ('Bond', 'ComplexBond')"

compare_hard_predicate = f"Sum(Results[DerivedValuation/{scope}/pv_x_modified_duration]) / Sum(Results[Valuation/PvInPortfolioCcy]) lt {duration_limit_threshold}"
compare_soft_predicate = f"Sum(Results[DerivedValuation/{scope}/pv_x_modified_duration]) / Sum(Results[Valuation/PvInPortfolioCcy]) lt {duration_warning_threshold}"

##### Define the Compliance Rule
upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code='portfolio_duration_lte_8yrs'),
    name='Portfolio Duration Less than 8 yrs',
    description='Check to ensure that the wtd average portfolio duration is less than 8 yrs',
    
    # Connect to the previously defined template and variation
    template_id=lm.ResourceId(scope=scope, code=f'{scope}-portfolio_duration_check'),
    variation=f"{scope}-portfolio_duration_check",
    
    portfolio_group_id = lm.ResourceId(scope = scope, code = portfolio_group_code),
    active = True,

    # Define values for each step in the template
    parameters = {
        "InstrumentTypeFilter.Predicate": lm.FilterPredicateComplianceParameter(
            value= instrument_type_filter_predicate,
            compliance_parameter_type='FilterPredicateComplianceParameter'),

        "PortfolioIdGroup.GroupingKey": lm.GroupBySelectorComplianceParameter(
            value= portfolio_grouping_key,
            compliance_parameter_type='GroupBySelectorComplianceParameter'),

        "Compare.HardPredicate": lm.GroupFilterPredicateComplianceParameter(
            value= compare_hard_predicate,
            compliance_parameter_type='GroupFilterPredicateComplianceParameter'),

        "Compare.SoftPredicate": lm.GroupFilterPredicateComplianceParameter(
            value= compare_soft_predicate,
            compliance_parameter_type='GroupFilterPredicateComplianceParameter')
    },
    
    properties = {}  # Optional metadata
)

##### Submit the Rule to LUSID
compliance_api.upsert_compliance_rule(upsert_compliance_rule_request = upsert_compliance_rule_request)

print("Rule created")

## 6. Run Post-Trade Compliance and Analyze Results

This section runs the compliance rules that we previously defined against the selected portfolio. We will then retrieve and analyze the results—first at the summary (rule-level) and then by drilling down into rule-specific details. 

**APIs & Methods Used:**
- `ComplianceApi.run_compliance_portfolio_rules(scope, code, request)`
- `ComplianceApi.get_compliance_run_by_id(id)` — To retrieve the results of a specific run.

In [ ]:
#### This initiates a compliance evaluation using the defined scope and valuation recipe.

run_response = compliance_api.run_compliance(
    run_scope=scope,
    rule_scope=scope,
    is_pre_trade=False,
    recipe_id_scope=scope,
    recipe_id_code="demo_valuation_recipe_mid"
)

# Capture the unique run identifier
run_code = run_response.run_id.code

print(f"Compliance run {run_response.run_id.scope}/{run_response.run_id.code} completed.")
print(f"Initiated: {run_response.instigated_at}, completed: {run_response.completed_at}")

### Summarize Rule-Level Results

In [ ]:
"""
This function generates a summary DataFrame showing each rule's:
- Status (OK, Warning, Failed)
- Number of affected orders or portfolios
"""

def rule_level_dataframe(run_summary):
    h = ["", "", "", "", ""]
    c = ["Rule", "Rule Name","Rule Description", "Status", "Affected Portfolios"]
    df = pd.DataFrame([c], columns=h)
    new_labels = pd.MultiIndex.from_arrays([df.columns, df.iloc[0]], names=["", ""])
    df = df.set_axis(new_labels, axis=1).iloc[1:]

    for d in run_summary.details:
        df.loc[len(df)] = [
            f"{d.rule_id.scope}/{d.rule_id.code}",
            d.rule_name,
            d.rule_description,
            d.status,
            len(d.affected_portfolios_details),
        ]

    return df

# Fetch decorated run summary containing all rule outcomes
run_summary = compliance_api.get_decorated_compliance_run_summary(
    scope=scope,
    code=run_code
)

# Generate and display rule summary table
df_summary = rule_level_dataframe(run_summary)
df_summary.columns = df_summary.columns.get_level_values(1)
print(f"Rule-level results for run {run_summary.run_id.scope}/{run_summary.run_id.code}.")
df_summary = df_summary.sort_values('Rule').reset_index(drop=True)
df_summary                  

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

### Drilldown – Portfolio Weighted Average Credit Rating Check

In [ ]:
# Fetch detailed results for rule2 (Average Credit Rating check)
rule_result_avg_credit = compliance_api.get_compliance_rule_result(
    run_scope=scope,
    run_code=run_code,
    rule_scope=scope,
    rule_code="portfolio_credit_rating_gte_A"
)

# Retrieve all rule breakdowns (no filtering on status)
filtered_breakdowns_avg_credit = [
    breakdown for breakdown in rule_result_avg_credit.rule_result.rule_breakdown
]

# Convert to DataFrame and calculate average numerical rating
rule_result_avg_credit_df = lusid_response_to_data_frame(filtered_breakdowns_avg_credit)
avg_credit_columns = ['lineage.1.subLabel','groupStatus', 'resultsUsed.DerivedValuation/compliance_demo/pv_x_numerical_credit_rating', 'resultsUsed.Valuation/PvInPortfolioCcy']
df_rule_summary_avg_credit = rule_result_avg_credit_df[avg_credit_columns].rename(columns={'lineage.1.subLabel' : 'PortfolioID', 'resultsUsed.DerivedValuation/compliance_demo/pv_x_numerical_credit_rating': 'pv_x_numerical_credit_rating', 'resultsUsed.Valuation/PvInPortfolioCcy': 'portfolio_pv'})
df_rule_summary_avg_credit['Wtd_Avg_Numerical_Rating (A Rating <= 6)'] = df_rule_summary_avg_credit['pv_x_numerical_credit_rating']/df_rule_summary_avg_credit['portfolio_pv']
df_rule_summary_avg_credit


### Drilldown – Maximum Exposure to High Yield Credit Rating Check

In [ ]:
# Fetch detailed results for rule2 (Average Credit Rating check)
rule_result_hy = compliance_api.get_compliance_rule_result(
    run_scope=scope,
    run_code=run_code,
    rule_scope=scope,
    rule_code="minimum_BB_rated_limit"
)

# Retrieve all rule breakdowns (no filtering on status)
filtered_breakdowns_hy = [
    breakdown for breakdown in rule_result_hy.rule_result.rule_breakdown
]

# Convert to DataFrame and calculate average numerical rating
rule_result_hy_df = lusid_response_to_data_frame(filtered_breakdowns_hy)

columns_hy = ['lineage.1.subLabel', 'groupStatus', 'resultsUsed.Valuation/PvInPortfolioCcy', 'resultsUsed.PortfolioIdGroup.Valuation/PvInPortfolioCcy']
df_rule_summary_hy = rule_result_hy_df[columns_hy].rename(columns={'lineage.1.subLabel': 'PortfolioID', 'resultsUsed.Valuation/PvInPortfolioCcy': 'pv_BB_or_lower', 'resultsUsed.PortfolioIdGroup.Valuation/PvInPortfolioCcy': 'portfolio_pv'})
df_rule_summary_hy['BB_or_lower_pct'] = df_rule_summary_hy['pv_BB_or_lower'] / df_rule_summary_hy['portfolio_pv']*100

df_rule_summary_hy

### Drilldown – Portfolio Weighted Average Duration Check

In [ ]:
# Fetch detailed results for rule2 (Average Credit Rating check)
rule_result_avg_duration = compliance_api.get_compliance_rule_result(
    run_scope=scope,
    run_code=run_code,
    rule_scope=scope,
    rule_code="portfolio_duration_lte_8yrs"
)

# Retrieve all rule breakdowns (no filtering on status)
filtered_breakdowns_avg_duration = [
    breakdown for breakdown in rule_result_avg_duration.rule_result.rule_breakdown
]

# Convert to DataFrame and calculate average numerical rating
rule_result_avg_duraction_df = lusid_response_to_data_frame(filtered_breakdowns_avg_duration)
columns_avg_duration = ['lineage.2.subLabel', 'groupStatus', 'resultsUsed.DerivedValuation/compliance_demo/pv_x_modified_duration', 'resultsUsed.Valuation/PvInPortfolioCcy']
df_rule_summary_avg_duration = rule_result_avg_duraction_df[columns_avg_duration].rename(columns={'lineage.2.subLabel': 'PortfolioID', 'resultsUsed.DerivedValuation/compliance_demo/pv_x_modified_duration': 'pv_x_modified_duration', 'resultsUsed.Valuation/PvInPortfolioCcy': 'portfolio_pv'})
df_rule_summary_avg_duration['Wtd_Avg_Portfolio_Duration'] = df_rule_summary_avg_duration['pv_x_modified_duration'] / df_rule_summary_avg_duration['portfolio_pv']

df_rule_summary_avg_duration

### Drilldown – Issuer Concentration Check

In [ ]:
# Fetch detailed results for rule1 (Simple PV threshold)
rule_result_issuer = compliance_api.get_compliance_rule_result(
    run_scope=scope,
    run_code=run_code,
    rule_scope=scope,
    rule_code="single_issuer_10pct_max"
)

# Filter to only show failed or warning results
filtered_breakdowns_issuer = [
    breakdown
    for breakdown in rule_result_issuer.rule_result.rule_breakdown
#    if getattr(breakdown, "group_status", "") in ("Failed", "Warning")
]

# Convert to DataFrame for display
rule_result_issuer_df = lusid_response_to_data_frame(filtered_breakdowns_issuer)

columns_issuer = ['lineage.1.subLabel', 'groupStatus', 'lineage.2.subLabel', 'resultsUsed.percent', 'resultsUsed.numerator', 'resultsUsed.denominator']
df_rule_summary_issuer = rule_result_issuer_df[columns_issuer].rename(columns={'lineage.1.subLabel' : 'PortfolioID', 'lineage.2.subLabel': 'UltimateParent_ID', 'resultsUsed.percent': '%_Held', 'resultsUsed.numerator': 'UltimateParent_MktValue', 'resultsUsed.denominator': 'Portfolio_MktValue'})

# Define the custom sort order
status_order = ['Failed', 'Warning', 'Passed']

# Convert the column to categorical with the specified order
df_rule_summary_issuer['groupStatus'] = pd.Categorical(df_rule_summary_issuer['groupStatus'], categories=status_order, ordered=True)

# Sort by the column
df_rule_summary_issuer = df_rule_summary_issuer.sort_values('groupStatus')

df_rule_summary_issuer


### Drilldown – Only US Dollar Denominated Holdings

In [ ]:
# Fetch detailed results for rule2 (Average Credit Rating check)
rule_result_usd = compliance_api.get_compliance_rule_result(
    run_scope=scope,
    run_code=run_code,
    rule_scope=scope,
    rule_code="usd_denominated_only"
)

# Retrieve all rule breakdowns (no filtering on status)
filtered_breakdowns_usd = [
    breakdown for breakdown in rule_result_usd.rule_result.rule_breakdown
]

# Convert to DataFrame and calculate average numerical rating
rule_result_usd_df = lusid_response_to_data_frame(filtered_breakdowns_usd)

columns_usd = ['lineage.1.subLabel', 'groupStatus', 'lineage.5.subLabel', 'lineage.4.subLabel', 'propertiesUsed.Holding/default/Currency.0.value.labelValue']
df_rule_summary_usd = rule_result_usd_df[columns_usd].rename(columns={'lineage.1.subLabel' : 'PortfolioID','lineage.5.subLabel': 'Instrument_Name', 'lineage.4.subLabel': 'ISIN', 'propertiesUsed.Holding/default/Currency.0.value.labelValue': 'Currency'})
df_rule_summary_usd



## 7. Create Simple Orders and Run Pre-Trade Compliance

In this final section, we simulate trading activity on the portfolio by submitting new buy/sell orders—then immediately perform a pre-trade compliance check to validate whether these proposed changes would cause any regulatory or policy breaches.

### Load and Submit Trade Orders

In [ ]:
# Load new trade orders from a CSV file
orders_df = pd.read_csv("data/demo-live_orders.csv")
orders_df

In [ ]:
# Prepare and submit orders using the LUSID Order API
order_requests = defaultdict(list)
order_sets = defaultdict(list)
responses = []

for index, order in orders_df.iterrows():

    request = lm.OrderRequest(
        id=lm.ResourceId(scope=scope, code=order["order_id"]),
        quantity=order["quantity"],
        side=order["side"],  # Buy/Sell
        instrument_identifiers={
            "Instrument/default/Isin": order["isin"],
            "Instrument/default/Figi": order["figi"],
        },
        properties={},
        portfolio_id=lm.ResourceId(scope=scope, code=portfolio_code),
        state=order["state"],
        type=order["type"],
        price=lm.CurrencyAndAmount(
            amount=0 if pd.isna(order["price"]) else order["price"],
            currency=order["currency"],
        ),
        limit_price=(
            lm.CurrencyAndAmount(
                amount=order["limit_price"], currency=order["limit_currency"]
            )
            if not pd.isna(order["limit_price"])
            and not pd.isna(order["limit_currency"])
            else None
        ),
        stop_price=(
            lm.CurrencyAndAmount(
                amount=order["stop_price"], currency=order["stop_currency"]
            )
            if not pd.isna(order["stop_price"]) and not pd.isna(order["stop_currency"])
            else None
        ),
    )

    request = lm.OrderSetRequest(order_requests=[request])
    response = orders_api.upsert_orders(order_set_request=request)
    responses.append(response.values[0])

# Summarize submitted orders for validation
attributes = [
    (
        o.id.code,
        o.instrument_identifiers["Instrument/default/Isin"],
        o.lusid_instrument_id,
        o.side,
        o.type,
        o.state,
        o.quantity,
        o.price.amount,
        o.price.currency,
        o.limit_price.amount if o.limit_price is not None else "N/A",
        o.limit_price.currency if o.limit_price is not None else "N/A",
    )
    for o in responses
]

pd.DataFrame(
    attributes,
    columns=[
        "order_id",
        "isin",
        "lusid_instrument_id",
        "side",
        "type",
        "state",
        "quantity",
        "price",
        "currency",
        "lim px",
        "lim ccy",
    ],
).style.format({"quantity": "{:20,.0f}", "price": "{:20,.2f}"})

### Run Pre-Trade Compliance

In [ ]:
# Run pre-trade compliance check based on new orders
order_run = compliance_api.run_compliance(
    is_pre_trade=True,
    recipe_id_scope=scope,
    recipe_id_code="demo_valuation_recipe_mid",
    run_scope=scope,
    rule_scope=scope,
)
pre_trade_run_code_order = order_run.run_id.code

In [ ]:
# Fetch decorated run summary containing all rule outcomes
pre_trade_run_summary = compliance_api.get_decorated_compliance_run_summary(
    scope=scope,
    code=pre_trade_run_code_order
)

# Capture the unique run identifier
pre_trade_run_code = pre_trade_run_summary.run_id.code

# Generate and display rule summary table
pre_trade_df_summary = rule_level_dataframe(pre_trade_run_summary)
pre_trade_df_summary.columns = pre_trade_df_summary.columns.get_level_values(1)
print(f"Rule-level results for run {pre_trade_run_summary.run_id.scope}/{pre_trade_run_summary.run_id.code}.")
pre_trade_df_summary = pre_trade_df_summary.sort_values('Rule').reset_index(drop=True)
pre_trade_df_summary  

### Pre-Trade Check - High Yield Exposure

In [ ]:
# Fetch detailed results for rule2 (Average Credit Rating check)
pre_trade_rule_result_hy = compliance_api.get_compliance_rule_result(
    run_scope=scope,
    run_code=pre_trade_run_code,
    rule_scope=scope,
    rule_code="minimum_BB_rated_limit"
)

# Retrieve all rule breakdowns (no filtering on status)
pre_trade_filtered_breakdowns_hy = [
    breakdown for breakdown in pre_trade_rule_result_hy.rule_result.rule_breakdown
]

# Convert to DataFrame and calculate average numerical rating
pre_trade_rule_result_hy_df = lusid_response_to_data_frame(pre_trade_filtered_breakdowns_hy)

pre_trade_columns_hy = ['lineage.1.subLabel', 'groupStatus', 'resultsUsed.Valuation/PvInPortfolioCcy', 'resultsUsed.PortfolioIdGroup.Valuation/PvInPortfolioCcy']
df_pre_trade_rule_summary_hy = pre_trade_rule_result_hy_df[pre_trade_columns_hy].rename(columns={'lineage.1.subLabel': 'PortfolioID', 'resultsUsed.Valuation/PvInPortfolioCcy': 'pv_BB_or_lower', 'resultsUsed.PortfolioIdGroup.Valuation/PvInPortfolioCcy': 'portfolio_pv'})
df_pre_trade_rule_summary_hy['BB_or_lower_pct'] = df_pre_trade_rule_summary_hy['pv_BB_or_lower'] / df_pre_trade_rule_summary_hy['portfolio_pv']*100
df_pre_trade_rule_summary_hy

### Pre-Trade Check - Wtd Avg Portfolio Credit Rating

In [ ]:
# Fetch detailed results for rule2 (Average Credit Rating check)
pre_trade_result_avg_credit = compliance_api.get_compliance_rule_result(
    run_scope=scope,
    run_code=pre_trade_run_code,
    rule_scope=scope,
    rule_code="portfolio_credit_rating_gte_A"
)

# Retrieve all rule breakdowns (no filtering on status)
pre_trade_filtered_breakdowns_avg_credit = [
    breakdown for breakdown in pre_trade_result_avg_credit.rule_result.rule_breakdown
]

# Convert to DataFrame and calculate average numerical rating
pre_trade_result_avg_credit_df = lusid_response_to_data_frame(pre_trade_filtered_breakdowns_avg_credit)
pre_trade_avg_credit_columns = ['lineage.1.subLabel','groupStatus', 'resultsUsed.DerivedValuation/compliance_demo/pv_x_numerical_credit_rating', 'resultsUsed.Valuation/PvInPortfolioCcy']
pre_trade_df_rule_summary_avg_credit = pre_trade_result_avg_credit_df[pre_trade_avg_credit_columns].rename(columns={'lineage.1.subLabel' : 'PortfolioID', 'resultsUsed.DerivedValuation/compliance_demo/pv_x_numerical_credit_rating': 'pv_x_numerical_credit_rating', 'resultsUsed.Valuation/PvInPortfolioCcy': 'portfolio_pv'})
pre_trade_df_rule_summary_avg_credit['Wtd_Avg_Numerical_Rating (A Rating <= 6)'] = pre_trade_df_rule_summary_avg_credit['pv_x_numerical_credit_rating']/pre_trade_df_rule_summary_avg_credit['portfolio_pv']
pre_trade_df_rule_summary_avg_credit

### Pre-Trade Check - Wtd Avg Portfolio Duration

In [ ]:
# Fetch detailed results for rule2 (Average Credit Rating check)
rule_result_avg_duration = compliance_api.get_compliance_rule_result(
    run_scope=scope,
    run_code=pre_trade_run_code,
    rule_scope=scope,
    rule_code="minimum_BB_rated_limit"
)

# Retrieve all rule breakdowns (no filtering on status)
filtered_breakdowns_avg_duration = [
    breakdown for breakdown in rule_result_avg_duration.rule_result.rule_breakdown
]

# Convert to DataFrame and calculate average numerical rating
rule_result_avg_duration_df = lusid_response_to_data_frame(filtered_breakdowns_avg_duration)

columns_avg_duration = ['lineage.1.subLabel', 'groupStatus', 'resultsUsed.Valuation/PvInPortfolioCcy', 'resultsUsed.PortfolioIdGroup.Valuation/PvInPortfolioCcy']
df_rule_summary_duration = rule_result_avg_duration_df[columns_avg_duration].rename(columns={'lineage.1.subLabel': 'PortfolioID', 'resultsUsed.Valuation/PvInPortfolioCcy': 'pv_BB_or_lower', 'resultsUsed.PortfolioIdGroup.Valuation/PvInPortfolioCcy': 'portfolio_pv'})
df_rule_summary_duration['BB_or_lower_pct'] = df_rule_summary_duration['pv_BB_or_lower'] / df_rule_summary_duration['portfolio_pv']*100

df_rule_summary_duration

### Pre-Trade Check - Single Issuer Concentration

In [ ]:
# Fetch detailed results for rule1 (Simple PV threshold)
pre_trade_result_issuer = compliance_api.get_compliance_rule_result(
    run_scope=scope,
    run_code=pre_trade_run_code,
    rule_scope=scope,
    rule_code="single_issuer_10pct_max"
)

# Filter to only show failed or warning results
pre_trade_filtered_breakdowns_issuer = [
    breakdown
    for breakdown in pre_trade_result_issuer.rule_result.rule_breakdown
]

# Convert to DataFrame for display
pre_trade_result_issuer_df = lusid_response_to_data_frame(pre_trade_filtered_breakdowns_issuer)

pre_trade_columns_issuer = ['lineage.1.subLabel', 'groupStatus', 'lineage.2.subLabel', 'resultsUsed.percent', 'resultsUsed.numerator', 'resultsUsed.denominator']
df_pre_trade_summary_issuer = pre_trade_result_issuer_df[pre_trade_columns_issuer].rename(columns={'lineage.1.subLabel' : 'PortfolioID', 'lineage.2.subLabel': 'UltimateParent_ID', 'resultsUsed.percent': '%_Held', 'resultsUsed.numerator': 'UltimateParent_MktValue', 'resultsUsed.denominator': 'Portfolio_MktValue'})

# Define the custom sort order
status_order = ['Failed', 'Warning', 'Passed']

# Convert the column to categorical with the specified order
df_pre_trade_summary_issuer['groupStatus'] = pd.Categorical(df_pre_trade_summary_issuer['groupStatus'], categories=status_order, ordered=True)

# Sort by the column
df_pre_trade_summary_issuer = df_pre_trade_summary_issuer.sort_values('groupStatus')

df_pre_trade_summary_issuer

### Pre-Trade Check - US Dollar 

In [ ]:
# Fetch detailed results for rule2 (Average Credit Rating check)
pre_trade_rule_result_usd = compliance_api.get_compliance_rule_result(
    run_scope=scope,
    run_code=pre_trade_run_code,
    rule_scope=scope,
    rule_code="usd_denominated_only"
)

# Retrieve all rule breakdowns (no filtering on status)
pre_trade_filtered_breakdowns_usd = [
    breakdown for breakdown in pre_trade_rule_result_usd.rule_result.rule_breakdown
]

# Convert to DataFrame and calculate average numerical rating
pre_trade_rule_result_usd_df = lusid_response_to_data_frame(pre_trade_filtered_breakdowns_usd)

pre_trade_columns_usd = ['lineage.1.subLabel', 'groupStatus', 'lineage.5.subLabel', 'lineage.4.subLabel', 'propertiesUsed.Holding/default/Currency.0.value.labelValue']
df_pre_trade_rule_summary_usd = pre_trade_rule_result_usd_df[pre_trade_columns_usd].rename(columns={'lineage.1.subLabel' : 'PortfolioID','lineage.5.subLabel': 'Instrument_Name', 'lineage.4.subLabel': 'ISIN', 'propertiesUsed.Holding/default/Currency.0.value.labelValue': 'Currency'})
df_pre_trade_rule_summary_usd